In [13]:
import os
import sys
import re
from argparse import Namespace

sys.path.append('../src-py/')

In [2]:
import torch
import json

import nltk
import numpy as np
import pandas as pd

from pathlib import Path
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration, BartModel

pd.set_option('display.max_colwidth', None)

In [3]:
bart_model = BartForConditionalGeneration.from_pretrained('../data/thirdparty/comet-atomic_2020_BART')
bart_tokenizer = BartTokenizer.from_pretrained('../data/thirdparty/comet-atomic_2020_BART')

In [23]:
input_ids = bart_tokenizer.encode('representative democracy', return_tensors='pt')
output = bart_model.generate(input_ids, max_length=50, num_beams=1, num_return_sequences=10, do_sample=True,top_k=100, temperature=0.9).numpy()

In [24]:
decoded_outputs = [bart_tokenizer.decode(x, skip_special_tokens=True) for x in output]
print(decoded_outputs)

[' the election being held', ' show democracy', ' use of ballot', ' selfgovernment', ' making of new', ' electable representatives', ' represent people', ' represent', 'representative democracy', ' use']


### Extract predicates from text:

In [11]:
kialo_train = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/kialo_data/kialo_train_df.pkl')

In [12]:
kialo_train[['conclusion_text', 'counter']].sample(10).head(n=10)

,conclusion_text,counter
33640,Humans should stop eating animal meat.,"The adoption of eating meat caused the human brain to grow and increase in power, leading to the modern human as we know it. Eating Meat Made Us Smarter- NPR"
40986,Liquid Democracy would be a better mechanism of governance than current representative democracy.,Misinformation will have a greater effect on regional/national policy.
7652,Kialo should separate impact voting into relevance and veracity of claim. TLDR,"Kialo has many development items on its list, this project may displace some that may be more important."
70184,Transgender people face a high risk of prejudice and mental health problems: approximately 41 percent of trans people in the U.S. have attempted suicide according to a 2010 survey by the National Center for Transgender Equality and the National Gay and Lesbian Task Force.,"This claim leaves out where the source says ""and mental-health problems"" after ""prejudice."" The suicide rate is a result of the mental-health problems. If it were a result of the prejudice, black people would have a higher suicide rate than white people. They don't"
71919,"Culture should, if necessary, be supported by the State.",Culture is not easily defined.
12817,"The Internet should be free \(as in free speech\), and the only practical way to keep it free is to treat every bit of the information transiting it — commercial or non-commercial — the same without regard to its content, author, buyer, or seller.",Bits are already not treated equally. Peering arrangements between companies/ISPs/services or content delivery networks already provide distinct advantages to data provision for companies that can afford them.
26043,Straight actors should not play gay characters.,"Gay people are not physically different than other people, so an heterosexual actor can seem as realistic on-screen as a gay one to play an homosexual character."
58015,Cultural appropriation is wrong.,"Adopting other cultures to simply enjoy \(e.g. listening to other cultures' music, celebrating their holidays, or enjoying their food\) is not harmful."
18309,Racially insensitive words in The Adventures of Huckleberry Finn should be changed when it's being taught in school.,Separating history from who we are makes meaningless our shared heritage. History is simply history and neither good nor bad. Perspective has to be maintained. Remembering slavery is vital to our shared history.
57614,Harry and Meghan no longer wish to take money from the royal family.,"The couple are seeking to trademark the Sussex Royal brand, thereby indirectly profiting from being royals."


In [14]:
sys.path.append('../../thirdparty/enhanced-subject-verb-object-extraction/')
from subject_verb_object_extract import findSVOs, nlp

In [15]:
kialo_sample = kialo_train.sample(50)

In [17]:
kialo_sample['conc_predicates'] = kialo_sample['conclusion_text'].apply(lambda x:  findSVOs(nlp(x)))
kialo_sample['counter_predicates'] = kialo_sample['counter'].apply(lambda x: findSVOs(nlp(x)))

In [20]:
kialo_sample[['conclusion_text', 'conc_predicates', 'counter', 'counter_predicates']].head(n=20)

,conclusion_text,conc_predicates,counter,counter_predicates
59240,Space tourism would create even more pollution.,"[(Space tourism, create, more pollution)]","This is avoided if we use hydrogen/oxygen fueled engine, which only produces water as a combustion by-product.","[(oxygen engine ,, use, we), (oxygen engine ,, fueled), (water, produce, which)]"
16777,The vegan agricultural industry is itself unethical too.,"[(The vegan agricultural industry, is, itself)]","If we act under capitalism, most of our actions will be unethical. Killing another living form for no real need is an unethical thing by itself.","[(we, act), (another form for real need, living)]"
57423,"Kialo should continue with just mobile browser support and not build a progressive web, IOS or Android app. \(Mobile browsers are fully-featured supported\)","[(Kialo, continue)]",A mobile app encourages more engagement.,"[(A mobile app, encourages, more engagement)]"
43737,Adoption is a more appropriate mechanism for childless families to have children.,"[(Adoption, is, a appropriate mechanism have children), (childless families, have, children)]","It is often difficult to adopt children due to restrictive laws, particularly governing older children.",[]
13786,"If God was really all powerful, he would be able to remove all evil.",[],"Omnipotence may not be intended as ""God can do everything"", but that God can do anything that is not excluded from HIS logic or from HIS nature.","[(Omnipotence, !intended), (HIS logic, !exclude, Omnipotence may be intended do ,)]"
33150,Even if protection was used one-night stands still have a non-neglible risk of sexually transmitted infections that could harm your significant other and mean they should be informed.,"[(protection, used), (infections, transmitted), (your significant other, harm, infections), (infections, mean), (they, informed)]","After a one-night stand, the cheating partner could get tested for STIs. If it turns out they have not caught an STI, then there is no risk.","[(STIs, test, the cheating partner), (it, turns), (an STI, !catch, they)]"
27801,Bullfighting encounters strong opposition from civil society.,"[(Bullfighting, encounters, strong opposition from civil society)]",Fans and members of the Catalan bullfighting federation \(FETC\) celebrated the overturning of the Catalan bullfighting ban. They expect the Monumental bullfighting arena to host the sport again soon.,"[(Fans, celebrated, the overturning of the Catalan bullfighting ban), (members of the Catalan bullfighting federation, celebrated, the overturning of), (They, expect), (the Monumental bullfighting arena, host, the sport)]"
19078,Religion has motivated numerous selfless acts such as volunteering and charity.,"[(Religion, motivated, numerous selfless acts as volunteering)]",Many seemingly or genuine selfless acts can cause more harm than good.,"[(Many seemingly selfless acts, cause, more harm than)]"
25878,The Bible has more bibliographic support than any other historical document of its time.,"[(The Bible, has, bibliographic support than any other historical document of its time)]","Iliad a text whose earliest copy is traced back to 900 B.C, has over 643 copies and each of them is 95% accurate in its text.","[(whose earliest copy, traced)]"
7799,Atheism holds that the absence of evidence for God's existence is good evidence that God does not exist.,"[(Atheism, holds), (God, !exist)]",Whether something is a good reason or not is subjective.,"[(something, is, a good reason)]"
